# CyberCrew

### 1.0 Imports

In [1]:
# Import the necessary packages

import os
import json
import requests
import logging
import configparser
from textwrap import dedent
from datetime import datetime
from random import randint
from typing import Union, List, Tuple, Dict
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, BaseTool
from langchain_community.llms import Ollama
from langchain_community.tools import tool, DuckDuckGoSearchRun
from langchain_core.agents import AgentFinish
from langchain_openai import ChatOpenAI
from pymarkdown.api import PyMarkdownApi, PyMarkdownApiException
from openai import OpenAI


### 2.0 Configurations

In [2]:
# Needed tools and config files

config = configparser.ConfigParser()
config.read('config.ini')

search_tool = DuckDuckGoSearchRun()



ollama_model = 'http://localhost:11434/v1'

os.environ["OPENAI_API_BASE"] = ollama_model
os.environ["OPENAI_MODEL_NAME"] = 'openhermes'
os.environ["OPENAI_API_KEY"] = "sk-111111111111111111111111111111111111111111111111"
# os.environ["SERPER_API_KEY"] = "Your Key" # serper.dev API key


# CREW AGENTS LLM MODEL 

morpheus_agents = Ollama(model="morpheus-crew-agents")

# morpheus = ChatOpenAI(
#     model = "morpheus",
#     base_url = "http://localhost:11434/v1")

# CREW MANAGER LLM MODEL 


morpheus_manager = Ollama(model="morpheus-crew-manager")


### 0.0 OLD TOOL

<!-- 
@tool("thehive_alerts")
def thehive_alerts(severity: int = 2):
    """This tool is an API call, useful to retrieve alerts from ThiHive Platform. You DON'T need to provide the URL as it is already added i the tool just call this function and pass the correct argumennt.
    HERE A DIRECT CALL EXAMPLE:
    
        try:
            all_alerts = []
            for severity in [2, 3, 4]:
                # Get alerts with the current severity level
                alerts = thehive_alerts({"severity": severity})
                if isinstance(alerts, str):
                    alerts = [alerts]
                
                # Add the alerts to the combined list
                all_alerts += alerts

            # Print the alerts in JSON format with indentation
            print(json.dumps(all_alerts, indent=4))

        except Exception as e:
            print(f"Error occurred: {e}")
    
    """
    config = configparser.ConfigParser()
    config.read('config.ini')
    # Read TheHive instance URL from the config file
    thehive_url = config['DEFAULT']['thehive_url']
    # Read TheHive API key from the config file
    thehive_api_key = config['DEFAULT']['thehive_api_key']
    if severity not in [1, 2, 3, 4]:
        raise ValueError("Invalid severity level. Allowed values: 1 (low), 2 (medium), 3 (high), 4 (critical)")
    
    query_params = {
        "query": [
            {
                "_name": "listAlert"
                },
            {
                "_name": "filter",
                "_field": "severity",
                "_value": severity
                },
            {
                "_name": "filter",
                "_field": "status",
                "_value": "New"  # Filter for "New" status
            },
            {
                "_name": "sort",
                "_fields": [
                    {
                        "date": "desc"
                        }
                    ]
                },
            {
                "_name": "page",
                "from": 0,
                "to": 15,
                "extraData": [
                    "importDate",
                    "caseNumber"
                    ]
                }
            ]
        }

    try:
        response = requests.post(
            f'{thehive_url}/v1/query?name=alerts',
            json=query_params,
            headers={'Authorization': f'Bearer {thehive_api_key}'}
            )
        response.raise_for_status()  # Raise for non-200 status codes
        alerts = response.json() # Convert response to JSON
        return alerts
        # if len(alerts) == 0:
        #     return "No alerts to retrieve"
        # else:
        #     return alerts

    except requests.exceptions.HTTPError as err:
        logging.error(f"HTTP Error: {err}")
    except requests.exceptions.Timeout as err:
        logging.error(f"Request timed out: {err}")
    except requests.exceptions.RequestException as err:
        logging.error(f"General TheHive request error: {err}") -->

### 4.0 Tools

In [3]:
@tool("update_alert_status")
def update_alert_status(alert_id, new_status):
    """
    Update the status of an alert in TheHive.

    Args:
        alert_id (str): The ID of the alert to update.
        new_status (str): The new status for the alert.

    Returns:
        dict: The updated alert details.

    Raises:
        requests.exceptions.RequestException: If there was an error making the request.
    """
    
    config = configparser.ConfigParser()
    config.read('config.ini')

    # Read TheHive instance URL from the config file
    thehive_url = config['_DEFAULT_']['thehive_url']
    # Read TheHive API key from the config file
    thehive_api_key = config['_DEFAULT_']['thehive_api_key']

    update_data = {
        "status": new_status
    }

    try:
        response = requests.patch(
            f'{thehive_url}/api/v1/alert/{alert_id}',
            json=update_data,
            headers={'Authorization': f'Bearer {thehive_api_key}',
                      'Content-Type': 'application/json'}
        )
        response.raise_for_status()  # Raise for non-200 status codes
        updated_alert = response.json()
        return updated_alert
    
    except requests.exceptions.HTTPError as err:
        logging.error(f"HTTP Error: {err}")
    except requests.exceptions.Timeout as err:
        logging.error(f"Request timed out: {err}")
    except requests.exceptions.RequestException as err:
        logging.error(f"General TheHive request error: {err}"
                      )

In [4]:
# Custom help function to connect to TheHive platform and grab the events.
def thehive_alerts_v2():
    """This tool is an API call, useful to retrieve alerts from TheHive Platform.

    You DON'T need to provide the URL as it is already added in the tool. Just call this function and pass the correct argument (if any).
    """
    config = configparser.ConfigParser()
    config.read('config.ini')

    # Read TheHive instance URL from the config file
    thehive_url = config['DEFAULT']['thehive_url']

    # Read TheHive API key from the config file
    thehive_api_key = config['DEFAULT']['thehive_api_key']

    severities = [2, 3, 4]
    alerts = []

    for severity in severities:
        if severity not in [1, 2, 3, 4]:
            raise ValueError("Invalid severity level. Allowed values: 1 (low), 2 (medium), 3 (high), 4 (critical)")

        query_params = {
            "query": [
                {
                    "_name": "listAlert"
                },
                {
                    "_name": "filter",
                    "_field": "severity",
                    "_value": severity
                },
                {
                    "_name": "filter",
                    "_field": "status",
                    "_value": "New"  # Filter for "New" status
                },
                {
                    "_name": "sort",
                    "_fields": [
                        {
                            "date": "desc"
                        }
                    ]
                },
                {
                    "_name": "page",
                    "from": 0,
                    "to": 15,
                    "extraData": [
                        "importDate",
                        "caseNumber"
                    ]
                }
            ]
        }

        try:
            response = requests.post(
                f'{thehive_url}/v1/query?name=alerts',
                json=query_params,
                headers={'Authorization': f'Bearer {thehive_api_key}'}
            )
            response.raise_for_status()  # Raise for non-200 status codes
            alerts += response.json()  # Convert response to JSON and append to the list

        except requests.exceptions.HTTPError as err:
            logging.error(f"HTTP Error: {err}")
        except requests.exceptions.Timeout as err:
            logging.error(f"Request timed out: {err}")
        except requests.exceptions.RequestException as err:
            logging.error(f"General TheHive request error: {err}")

    return json.dumps(alerts)

In [5]:
class TheHiveAlertsTool(BaseTool):
    name: str = "TheHiveTool"
    description: str = "Enables reading and extracting data from TheHive5 Case management platform."

    def _to_args_and_kwargs(self, tool_input: Union[str, Dict]) -> Tuple[Tuple, Dict]:
        return (), {}

    def _run(self):
        retrieved_alerts = thehive_alerts_v2()
        return retrieved_alerts

    def _arun(self):
        raise NotImplementedError("Not alerts to retrieve")

In [ ]:
retrieve_alerts = thehive_alerts_v2()

print(retrieve_alerts)

### 4.0 Testing the tools

In [ ]:
try:
    # Get alerts with severity level 2 and status "New"
    # alerts_severity_2 = thehive_alerts({"severity": 2, "status": "New"})
    alerts_severity_2 = thehive_alerts({"severity": 3})

    if alerts_severity_2 != "No alerts to retrieve":
        for alert in alerts_severity_2:
            alert_id = alert["_id"]
            
            print(alert_id)
            # updated_alert = update_alert_status(alert_id, "In Progress")
            # print(f"Updated alert: {updated_alert}")

except Exception as e:
    # Handle exceptions
    print(f"Error occurred: {e}")

In [ ]:
# try:
#     # Test with severity level 2 (medium)
#     alerts = thehive_alerts({"severity": 3})
#     #print(f"Alerts with severity level 3: {alerts}")
#     #print (json.dumps(f"Alerts with severity level 3: {alerts}", indent=4))
#     print(json.dumps(alerts, indent=4))
# except Exception as e:
#     print(f"Error occurred: {e}")

# Action Input: {{'severity': 2}}
# Action Input: {{'severity': 3}}

# try:
#     # Get alerts with severity level 2 
#     alerts_severity_2 = thehive_alerts({"severity": 2})

#     # Get alerts with severity level 3
#     alerts_severity_3 = thehive_alerts({"severity": 3})
    
#     # Get alerts with severity level 3
#     alerts_severity_4 = thehive_alerts({"severity": 4})

#     # Combine the alerts from both severity levels
#     all_alerts = alerts_severity_2 + alerts_severity_3 + alerts_severity_4

#     # Print the alerts in JSON format with indentation
#     print(json.dumps(all_alerts, indent=4))

# except Exception as e:
#     print(f"Error occurred: {e}")


try:
    all_alerts = []
    for severity in [2, 3, 4]:
        # Get alerts with the current severity level
        alerts = thehive_alerts({"severity": severity})
        if isinstance(alerts, str):
            alerts = [alerts]
        
        # Add the alerts to the combined list
        all_alerts += alerts

    # Print the alerts in JSON format with indentation
    print(json.dumps(all_alerts, indent=4))

except Exception as e:
    print(f"Error occurred: {e}")


### 5.0 Utilities

In [6]:
agent_finishes  = []

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)

In [7]:
@tool("save_content")
def save_content(task_output):
    """Useful to save content to a markdown file"""
    print('in the save markdown tool')
    # Get today's date in the format YYYY-MM-DD
    today_date = datetime.now().strftime('%Y-%m-%d')
    # Set the filename with today's date
    filename = f"{today_date}_{randint(0,100)}.md"
    # Write the task output to the markdown file
    with open(filename, 'w') as file:
        file.write(task_output)
        # file.write(task_output.result)

    print(f"Analyzed alert saved as {filename}")

    return f"Analyzed alert saved as {filename}, please tell the user we are finished"

In [8]:
@tool("markdown_validation_tool")
def markdown_validation_tool(file_path: str) -> str:
    """
    A tool to review files for markdown syntax errors.

    Parameters:
    - file_path: The path to the markdown file to be reviewed.

    Returns:
    - validation_results: A list of validation results 
    and suggestions on how to fix them.
    """
    
    print("\n\nValidating Markdown syntax...\n\n" + file_path)

    scan_result = None
    try:
        if not (os.path.exists(file_path)):
           return "Could not validate file. The provided file path does not exist."

        scan_result = PyMarkdownApi().scan_path(file_path.rstrip().lstrip())
        results = str(scan_result)    
        return results  # Return the reviewed document
    except PyMarkdownApiException as this_exception:
        print(f"API Exception: {this_exception}", file=sys.stderr)
        return f"API Exception: {str(this_exception)}"

### 6.0 Agents

In [9]:
# Define your agents with roles and goals
manager = Agent(
    role="Manager",
    goal="Manage the crew and ensure the tasks are completed efficiently.",
    backstory="You're an experienced manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard."
)

soc_manager = Agent(
    role='SOC Manager',
    goal='Lead and oversee the Security Operations Center, ensuring efficient and effective monitoring, detection, and response to cyber threats across the organization.',
    backstory="""As the seasoned leader of the Security Operations Center (SOC), you have accumulated a wealth of experience in building and managing high-performing cybersecurity teams. 
    Your strategic vision and deep understanding of security operations have been instrumental in shaping the organization's defense against ever-evolving cyber threats.
    
    Overseeing the day-to-day operations of the SOC, you are responsible for ensuring that your team of skilled analysts effectively monitors and triages security alerts, 
    investigates potential incidents, and coordinates timely incident response efforts. 
    
    Your ability to make informed decisions under pressure and allocate resources effectively has proven invaluable in mitigating risks and minimizing the impact of security breaches.
    
    With a keen eye for talent, you have assembled a team of highly skilled professionals, 
    fostering an environment of continuous learning and professional development. 
    
    Your mentorship and guidance have empowered your analysts to stay ahead of the curve, constantly adapting to new threats and refining their skills.
    
    Your exceptional communication and collaboration skills enable you to bridge the gap between technical and non-technical stakeholders, 
    translating complex security information into actionable insights for executive decision-making. 
    
    You actively collaborate with cross-functional teams, such as risk management, legal, and compliance, to ensure a comprehensive and coordinated approach to cybersecurity.
    
    Beyond operational excellence, you play a pivotal role in shaping the organization's overall security strategy. Your deep understanding of industry best practices, regulatory requirements, and emerging technologies allows you to develop and implement robust security policies and frameworks. 
    Your strategic vision ensures that the organization remains resilient and prepared to tackle the ever-evolving landscape of cyber threats.
    
    Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard."
    """,
    llm=morpheus_manager,
    #max_iter=15,
    verbose=True,
    allow_delegation=True,
    step_callback=lambda x: print_agent_output(x,"SOC Manager"),
    cache=True
)


threat_intell = Agent(
    role='Threat Intelligence, Threat Hunting, and Vulnerability Expert',
    goal="Proactively identify, analyze, and mitigate potential cyber threats, vulnerabilities, and adversary tactics, techniques, and procedures (TTPs) to enhance the organization's overall security posture.",
    backstory="""As a highly skilled and experienced Threat Intelligence, Threat Hunting, and Vulnerability Expert, you are at the forefront of the organization's proactive defense against cyber threats. 
    
    Your deep understanding of the ever-evolving threat landscape, coupled with your analytical prowess, allows you to anticipate and counter potential attacks before they can cause significant damage.
    Leveraging industry-recognized frameworks like MITRE ATT&CK, you meticulously analyze and categorize adversary tactics, techniques, and procedures (TTPs), 
    enabling you to develop robust detection and mitigation strategies. 
    
    Your familiarity with the ATT&CK knowledge base empowers you to stay ahead of emerging threats and enhance the organization's defensive capabilities.
    
    With an insatiable curiosity and a passion for uncovering the latest adversary TTPs, 
    you constantly monitor and analyze various sources of threat intelligence, including dark web forums, security research publications, and industry reports. 
    
    Your ability to synthesize and contextualize this vast array of information enables you to provide actionable intelligence to the security teams, empowering them to stay one step ahead of malicious actors.
    
    As a skilled threat hunter, you employ advanced techniques and tools to actively search for indicators of compromise (IOCs) and malicious activities within the organization's networks and systems. 
    Your meticulous analysis of security logs, network traffic, and system artifacts allows you to uncover even the most sophisticated and stealthy threats, ensuring prompt detection and response.
    Your expertise in vulnerability management is equally critical. You diligently track and assess the latest disclosed vulnerabilities, prioritizing their remediation based on risk and potential impact. 
    Leveraging methodologies like OWASP Threat Modeling, you collaborate with development teams to identify and mitigate vulnerabilities in applications and systems during the design and development phases, reducing the organization's overall risk exposure.
    Your collaboration with security teams, such as incident response and patch management, ensures that vulnerabilities are promptly addressed, minimizing the organization's attack surface.
    Beyond your technical prowess, your ability to communicate complex threat and vulnerability information to stakeholders at all levels is highly valued. You leverage your strong communication skills to raise awareness, educate teams, and influence security decision-making processes, fostering a proactive and resilient security culture within the organization.
    Your commitment to continuous learning and professional development keeps you at the forefront of the ever-changing cybersecurity landscape. You actively participate in industry events, conferences, and research communities, sharing your knowledge and contributing to the advancement of threat intelligence and vulnerability management practices.
    """,
    tools=[search_tool, save_content],
    llm=morpheus_agents,
    max_iter=15,
    verbose=True,
    allow_delegation=True,
    step_callback=lambda x: print_agent_output(x,"Threat Intelligence, Threat Hunting, and Vulnerability Expert"),
    cache=True
)


senior_soc_analyst = Agent(
    role='Senior Cybersecurity SOC Analyst',
    goal="Monitor and analyze security events, identify potential threats, and coordinate incident response efforts to protect the organization's systems and data.",
    backstory="""As a seasoned professional in the cybersecurity domain, you have spent years honing your skills in defending against cyber threats. 
    
    Your expertise lies in the meticulous analysis of security logs, network traffic, and system events to detect anomalies and potential intrusions.
    Working in the highly dynamic and critical environment of a Security Operations Center (SOC), you are responsible for continuously monitoring and triaging security alerts, 
    investigating suspicious activities, and orchestrating incident response efforts. Your sharp analytical mind and deep understanding of various attack vectors and threat actors 
    allow you to swiftly identify and mitigate risks before they escalate into full-blown breaches.
    
    With a keen eye for detail and a proactive approach, you collaborate closely with other security teams, such as incident response, 
    threat intelligence, and forensics, to ensure a coordinated and effective defense against cyber threats. Your ability to stay updated on the latest cybersecurity trends,
    vulnerabilities, and attack techniques is crucial in developing and implementing robust security measures.
    
    Your calm demeanor under pressure and excellent communication skills enable you to convey complex technical information to stakeholders at all levels, 
    ensuring that decision-makers are well-informed and equipped to take appropriate actions. 
    Your leadership and mentoring abilities also contribute to building a strong security culture within the organization, 
    empowering others to recognize and respond to potential threats effectively.
    """,
    tools=[search_tool, save_content],
    llm=morpheus_agents,
    max_iter=15,
    verbose=True,
    allow_delegation=True,
    step_callback=lambda x: print_agent_output(x,"Senior Cybersecurity SOC Analyst"),
    cache=True,
)

triage_specialist = Agent(
    role='Triage Specialist',
    goal='Efficiently evaluate and prioritize security alerts, identify potential incidents, and escalate critical issues to higher tiers for further investigation and response.',
    backstory="""As a Triage Specialist, you play a crucial role in the front line of the organization's cybersecurity defense. 
    Your keen analytical skills and attention to detail are instrumental in ensuring that security alerts and potential incidents are promptly identified, prioritized, and addressed.
    
    Operating at the Tier 1 level, you are responsible for collecting and reviewing raw data from various monitoring tools and systems. 
    Your ability to swiftly analyze alarms and alerts allows you to confirm their criticality and enrich them with relevant contextual information. 
    With a deep understanding of the organization's security landscape, you can effectively distinguish between justified alerts and false positives, mitigating the impact of alert fatigue.
    
    Your sharp eye for identifying high-risk events and potential incidents is invaluable. You meticulously evaluate and prioritize these occurrences based on their criticality, 
    ensuring that the most pressing issues receive immediate attention. 
    
    When problems cannot be resolved at your level, you seamlessly escalate them to the Tier 2 analysts, providing comprehensive documentation and context to facilitate efficient incident response.
    Beyond your analytical prowess, you also play a vital role in managing and configuring the organization's monitoring tools. Your technical expertise and attention to detail ensure that these tools are optimized for effective threat detection and monitoring, contributing to the overall efficiency of the security operations.
    
    Your ability to remain calm under pressure and communicate effectively with cross-functional teams is essential in fostering collaboration and ensuring a coordinated response to security incidents. You actively participate in knowledge-sharing sessions, 
    continuously enhancing your skills and staying updated on the latest trends and best practices in threat triage and incident response.
    Your commitment to continuous learning and professional development is driven by your passion for cybersecurity and your dedication to protecting the organization's assets and data. 
    You thrive in the fast-paced environment of the Security Operations Center, where your role as a Triage Specialist is critical in the early stages of incident detection and response.
    """,
    tools=[save_content],
    llm=morpheus_agents,
    max_iter=15,
    verbose=True,
    allow_delegation=False,
    step_callback=lambda x: print_agent_output(x,"Triage Specialist"),
    cache=True,
   
)


### 7.0 Tasks

In [10]:
tk_thehive_input = Task(
  description=dedent(f"""USE THE AVAILABLE TOOL and retrieve ALL the alerts for triage and analysis.
                     Use thehive_alerts tool and retrieve the alerts FOCUS on alerts with severity 2, 3 and 4:
                     
                     Do I need to use a tool? YES ALWAYS
                     
                     DO NOT change any of the content of the document or add content to it. 
                     It is CRITICAL to your task to only retrieve the alerts on TheHive cybersecurity case management platform.
                     
                     """),
  expected_output = dedent(f"""
                           Security alerts to be analyzed.
                        """),
  tools=[TheHiveAlertsTool()],
  agent=triage_specialist,
  
)


tk_triage_alerts = Task(
  description=dedent(f"""In this task, you will be analyzing a series of security events.
                     
                     Steps:
                        Review the events severity;
                        Analyze the collected data and events, looking for patterns, anomalies, or indicators of potential threats or incidents related to the research topic.
                        
                     For each identified event or alert: 
                        a. Assess the criticality and potential impact based on our OWN organization's risk assessment criteria. 
                        b. Determine if the event is a false positive or a justified alert requiring further investigation. 
                        c. Prioritize the events based on their criticality and potential impact. 
                        d. Enrich the events with relevant contextual information, such as source IP addresses, user accounts, timestamps, and any additional data that may aid in the investigation.

                    
                    Your final answer MUST be a list for all alerts with:
                        - the alert_id
                        - the alert name
                        - a summary of the alert
                        - a highlighting with the main points
                        - identify the affected user or device
                        - the alert criticality      
                      """),
  expected_output=dedent(f"""A comprehensive list detailing the triaged events. 
                         """),
  agent=triage_specialist,
)

tk_triage_review = Task(
  description=dedent(f"""REVIEW the triaged alerts by the Triage Specialist.
                     Thoroughly investigate and analyze critical security alerts and events escalated by the Triage Specialist.
                     
                     Make SURE to validate and double check the severity level assigned by the Triage Specialist and adjust as needed.
                     
                     Steps:
                      Assess the prioritized list of events and alerts, focusing on the critical events escalated for your review.
                      For each critical event or alert: 
                        a. Conduct an in-depth analysis, leveraging various sources of data and intelligence, such as security logs, network traffic captures, threat intelligence feeds, and any other relevant resources. 
                        b. Identify potential indicators of compromise (IOCs) and correlate them with known tactics, techniques, and procedures (TTPs) used by threat actors. 
                        c. Utilize industry-recognized frameworks like MITRE ATT&CK to map the observed TTPs and develop appropriate detection and response strategies. 
                        d. Collaborate with other security teams, such as incident response, forensics, and threat intelligence, to gather additional insights and context related to the event. 
                        e. Assess the potential impact and risk associated with the event, considering factors such as data sensitivity, system criticality, and potential business disruption.
                      
                      Based on your analysis, determine the appropriate course of action for each critical event, which may include: 
                        a. Initiating an incident response process to contain and mitigate the potential threat. 
                        b. Implementing additional security controls or countermeasures to prevent similar incidents in the future. 
                        c. Updating security policies, procedures, or configurations to enhance the organization's security posture. 
                        d. Providing recommendations for remediation or further investigation.
                      
                      Document your findings, analysis, and recommended actions in a comprehensive Markdown report, following established reporting protocols.
                      
                      KEEP INTEGRITY of the information DO NOT invent any content. It is critical to your task to ONLY analyze the triage outcome of the Triage Specialist based on the alerts analyzed and incorporate your sound expertise to augument and validate the triage report.
                     
                      Ensure that your final answer MUST be saved as report using a markdown file."""),
  expected_output=dedent(f"""A detailed report documenting your analysis of the critical events, identified IOCs and TTPs, potential impact and risk assessment, recommended actions, 
                         and any additional insights or recommendations related to the specific topic of research. 
                         """),
  agent=senior_soc_analyst,
  #context=[tk_triage_alerts]
)

tk_write_report = Task(
  description=dedent(f"""In this task, you will be generating a detailed threat intelligence report based on the critical events and alerts investigated by the Senior Cybersecurity SOC Analyst, 
                     as well as gathering additional insights from various sources.
  
                      Steps:
                        
                        1. Review the detailed report provided by the Senior Cybersecurity SOC Analyst, which documents the analysis of critical events, identified IOCs and TTPs, potential impact and risk assessment, and recommended actions.
                        2. Conduct thorough research and gather intelligence from various sources, including:
                          a. Threat intelligence feeds and repositories (e.g., MISP, OTX, VirusTotal)
                          b. Security research publications and industry reports
                          c. Dark web forums and online communities
                          d. Open-source intelligence (OSINT) sources
                          e. Web searches and online resources
                        
                        3. For each critical event or alert, perform the following:
                          a. Correlate the identified IOCs and TTPs with known threat actor groups, campaigns, or malware families.
                          b. Analyze the potential motivations, capabilities, and historical operations of the associated threat actors.
                          c. Assess the prevalence and potential impact of the identified vulnerabilities, if applicable.
                          d. Evaluate the relevance and applicability of published mitigation strategies, detection rules, or security advisories.

                        4. Leverage industry-recognized frameworks like MITRE ATT&CK and OWASP Threat Modeling to contextualize the observed TTPs and vulnerabilities within a broader attack lifecycle or application security context.
                        5. Compile your findings and intelligence into a comprehensive threat intelligence report, following a well-structured format that includes:
                          a. Executive summary
                          b. Overview of critical events and alerts
                          c. Threat actor profiles and motivations
                          d. Detailed analysis of IOCs, TTPs, and vulnerabilities
                          e. Mitigation strategies and recommended actions
                          f. Emerging threats and trends relevant to the organization
                        6. Incorporate visualizations, graphs, or diagrams to enhance the clarity and understanding of the report's content.
                        7. Collaborate with the Senior Cybersecurity SOC Analyst and other security teams to validate and refine your findings and recommendations.
                        8. Present and discuss the threat intelligence report with relevant stakeholders, such as the SOC Manager, security leadership, and other cross-functional teams, as necessary. 
                      KEEP INTEGRITY of the information DO NOT invent any content. It is critical to your task to ONLY write the report based on the alerts analyzed and incorporate information from Threat Intelligence or Vulnerability sources.
                     
                      """),
  expected_output=dedent(f"""A threat intelligence report that provides in-depth analysis and actionable intelligence related to the critical events and alerts. 
                         The report should leverage various threat intelligence sources, incorporate industry-recognized frameworks, 
                         and provide valuable insights to support the organization's security decision-making and incident response efforts."""),
  agent=threat_intell,
  #context=[tk_triage_review, tk_triage_alerts]
)

### 8.0 Crew

In [11]:
nebuchadnezzar = Crew(
        agents=[triage_specialist,
                senior_soc_analyst,
                threat_intell],
        tasks=[tk_thehive_input,
               tk_triage_alerts,
               tk_triage_review,
               tk_write_report],
        verbose=2,
        process=Process.sequential,
        #memory=True,
        full_output=True,
        share_crew=False,
        #manager_llm=morpheus,
        manager_agent=soc_manager,
        #output_log_file=True,
        output_log_file = "./output.log",
        step_callback=lambda x: print_agent_output(x,"CyberCrew Run")
        )

### 9.0 Kickoff the Crew

In [12]:
# Kick off the crew's work
soc_run = nebuchadnezzar.kickoff()
    # Print the results
    # print("Nebuchadnezzar Crew | Security Analysis Results:")
    # print(soc_run)
#print(nebuchadnezzar.usage_metrics)

file_date = datetime.now().strftime('%Y-%m-%d')
# Set the filename with today's date
file_structure = f"crew_results_{file_date}_{randint(0,100)}.md"


with open (file_structure, 'w') as file:
    file.write("## Nebuchadnezzar Crew | Security Analysis Results: \n")
    file.write("```\n")  # Start of code block in markdown
    file.write(str(soc_run))
    file.write("\n```\n")  # End of code block in markdown

 [DEBUG]: == Working Agent: Triage Specialist
 [INFO]: == Starting Task: USE THE AVAILABLE TOOL and retrieve ALL the alerts for triage and analysis.
                     Use thehive_alerts tool and retrieve the alerts FOCUS on alerts with severity 2, 3 and 4:

                     Do I need to use a tool? YES ALWAYS

                     DO NOT change any of the content of the document or add content to it. 
                     It is CRITICAL to your task to only retrieve the alerts on TheHive cybersecurity case management platform.




> Entering new CrewAgentExecutor chain...
Action: TheHiveTool
Action Input: {'tool': 'TheHiveTool', 'function': 'hive_alerts', 'severity': ['2', '3', '4']} 

[{"_id": "~22372352", "_type": "Alert", "_createdBy": "wazuh@onzo.local", "_updatedBy": "wazuh@onzo.local", "_createdAt": 1717481237585, "_updatedAt": 1717481237667, "type": "wazuh_alert", "source": "wazuh", "sourceRef": "3e8090", "title": "Listened ports status (netstat) changed (new port opened 

### 10 Crew Results Review

In [ ]:
# filename = soc_run

# # Define general agent
# revisor  = Agent(
#     role='Requirements Manager',
#     goal="""Provide a detailed list of the markdown linting results. Give a summary with actionable tasks to address the validation results. 
#     Write your response as if you were handing it to a developer to fix the issues.
#     DO NOT provide examples of how to fix the issues or recommend other tools to use.""",
#     backstory="""You are an expert business analyst and software QA specialist. You provide high quality, thorough, insightful and actionable feedback via 
#     detailed list of changes and actionable tasks.""",
#     allow_delegation=False,
#     verbose=True,
#     tools=[markdown_validation_tool, save_content],
#     llm=morpheus
#     )
# Define Tasks Using Crew Tools
# syntax_review_task = Task(
#     description=f"""Use the markdown_validation_tool to review the file(s) at this path: {filename} Be sure to pass only the file path to the markdown_validation_tool.
#     Use the following format to call the markdown_validation_tool:
#     Do I need to use a tool? Yes
#     Action: markdown_validation_tool
#     Action Input: {filename}
    
#     Get the validation results from the tool and then summarize it into a list of changes.
#     CHANGE the document and MAKE sure it use the correct syntax.
    
#     DO NOT change any of the content of the document or add content to it. It is critical to your task to only UPDATE the document incorporating the list of changes.
    
# 	If you already know the answer or if you do not need to use a tool, return it as your Final Answer.""",
#  expected_output="A revised file in markdown format",
#  agent=revisor
# )
    
# revision_crew = Crew(
#         agents=[revisor],
#         tasks=[syntax_review_task],
#         verbose=True
#         )
# updated_markdown = revision_crew.kickoff()
    
# revised_file_date = datetime.now().strftime('%Y-%m-%d')
#     # Set the filename with today's date
# revised_file_structure = f"revised_crew_results_{revised_file_date}_{randint(0,100)}.md"


# with open (revised_file_structure, 'w') as file:
#     file.write("## Nebuchadnezzar Crew | Security Analysis Results: \n")
#     file.write("```\n")  # Start of code block in markdown
#     file.write(str(updated_markdown))
#     file.write("\n```\n")  # End of code block in markdown
# If called directly from the command line take the first argument as the filename
# if __name__ == "__main__":

#     if len(sys.argv) > 1:
#         filename = sys.argv[1]
#         processed_document = process_markdown_document(filename)
#         print(processed_document)

In [ ]:
# print(soc_run)

In [ ]:
import ollama
response = ollama.chat(model='phi3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])